In [1]:
import os
import json
import traceback
import pandas as pd
from dotenv import load_dotenv
import PyPDF2

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

In [3]:
load_dotenv()
key=os.getenv("OPENAI_API_KEY")
print(key)

sk-proj-8BBGlF0eEGn28EslYqm1T3BlbkFJPXX96iyA14fveldJzZ3I


In [4]:
from langchain.chat_models import ChatOpenAI

In [5]:
llm=ChatOpenAI(openai_api_key=key,model="gpt-3.5-turbo")
# ,max_tokens=100

c:\Users\parte\Desktop\GenAI\Langchain\MCQGEN\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
Template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice  
questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format you response like RESPONSE_JSON below and use it as a guide. 
Ensure to make {number} questions.
### RESPONSE_JSON
{response_json}
"""

In [8]:
mcq_gen_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=Template
)

In [9]:
mcq_gen_chain=LLMChain(llm=llm,prompt=mcq_gen_prompt,output_key="quiz",verbose=True)

In [10]:
Template_2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
mcq_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=Template_2
    )

In [12]:
mcq_eval_chain=LLMChain(llm=llm, prompt=mcq_evaluation_prompt, output_key="review", verbose=True)

In [13]:
gen_eval_chain=SequentialChain(chains=[mcq_gen_chain,mcq_eval_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [14]:
with open("data.txt") as f:
    text=f.read()

In [15]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [16]:
TEXT=text
NUMBER=3
SUBJECT="Machine Learning"
TONE="Medium"

In [17]:
with get_openai_callback() as cb:
    response=gen_eval_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\parte\Desktop\GenAI\Langchain\MCQGEN\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were later found to be reinventions of the generalized linear models of statistics.[23] Probabilistic reasoning was also employed, especially in automated medical diagnosis.[24]:â€Š488â€Š

However, an increasing emphasis on the logical, knowledge-based approach caused a rift between AI and machine learning. Probabilistic systems were plagued by theoretical and practical problems of data acquisition and representation.[24]:â€Š488â€Š By 1980, expert systems had come to dominate AI, and statistics

In [18]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2384
Prompt Tokens:2012
Completion Tokens:372
Total Cost:0.0037620000000000006


In [22]:
quiz=json.loads(response.get("quiz"))
quiz


{'1': {'mcq': 'What was the main goal of machine learning in the early days of AI as an academic discipline?',
  'options': {'a': 'Achieving artificial intelligence',
   'b': 'Tackling solvable problems of a practical nature',
   'c': 'Focusing on data compression techniques',
   'd': 'Dominating the field of statistics'},
  'correct': 'a'},
 '2': {'mcq': 'What is the main difference between machine learning and data mining?',
  'options': {'a': 'Machine learning focuses on the discovery of unknown properties in the data, while data mining focuses on prediction.',
   'b': 'Data mining uses machine learning methods as a preprocessing step, while machine learning does not.',
   'c': 'Machine learning evaluates performance based on the ability to reproduce known knowledge, while data mining focuses on discovering previously unknown knowledge.',
   'd': 'Data mining employs supervised learning methods, while machine learning uses unsupervised learning.'},
  'correct': 'c'},
 '3': {'mcq': '

In [43]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [44]:
quiz_df=pd.DataFrame(quiz_table_data)
quiz_df

,MCQ,Choices,Correct
0,What was the main goal of machine learning in ...,a: Achieving artificial intelligence | b: Tack...,a
1,What is the main difference between machine le...,a: Machine learning focuses on the discovery o...,c
2,How is k-means clustering utilized in data com...,a: It groups similar data points into clusters...,b


In [45]:
quiz_df.to_csv("ML_Quiz.csv",index=False)